In [2]:
# Step 1: Import necessary libraries
# We need re for regular expressions, sympy for symbolic logic, itertools for product (truth tables),
# dataclasses for defining classes, and typing for type hints.
import re
from sympy import symbols, And, Or, Not, Implies, Equivalent, sympify
from itertools import product
from dataclasses import dataclass
from typing import Optional, List, Set, Dict, Tuple

# Step 2: Set up propositional logic mappings
# Define a dictionary mapping symbols to natural language phrases and its reverse.
prop_map: Dict[str, str] = {
    'P': 'it is raining',
    'Q': 'it is cold',
}
reverse_map: Dict[str, str] = {v: k for k, v in prop_map.items()}

# Step 3: Create symbolic variables and locals dictionary for sympy
# Use sympy symbols for the propositions and map logical operators.
_logic_symbols = symbols(' '.join(prop_map.keys()))
locals_dict: Dict[str, object] = dict(zip(prop_map.keys(), _logic_symbols))
locals_dict.update({
    '&': And,
    '|': Or,
    '~': Not,
    '->': Implies,
    '<->': Equivalent,
})

# Step 4: Define function to check if input is a logic expression
# This checks for operators or proposition symbols to determine input type.
def is_logic_input(input_str: str) -> bool:
    logic_ops = {'&', '|', '~', '->', '<->'}
    upper = input_str.upper()
    return any(op in input_str for op in logic_ops) or any(k in upper for k in prop_map.keys())

# Step 5: Define function to parse natural language sentence to logic string
# Convert sentence to lowercase, replace phrases with symbols, and handle connectives.
def parse_sentence_to_logic(s: str) -> str:
    s = s.lower().strip('.!? ')
    for phrase, sym in sorted(reverse_map.items(), key=lambda x: len(x[0]), reverse=True):
        s = re.sub(r'\b' + re.escape(phrase) + r'\b', sym, s)
    s = re.sub(r'\bnot\b\s+([A-Z]+)', r'~\1', s, flags=re.IGNORECASE)
    s = re.sub(r'([A-Z~]+)\s+\band\b\s+([A-Z~]+)', r'\1 & \2', s, flags=re.IGNORECASE)
    s = re.sub(r'([A-Z~]+)\s+\bor\b\s+([A-Z~]+)', r'\1 | \2', s, flags=re.IGNORECASE)
    s = re.sub(r'\bif\b\s+([A-Z~]+)\s+\bthen\b\s+([A-Z~]+)', r'\1 -> \2', s, flags=re.IGNORECASE)
    s = re.sub(r'([A-Z~]+)\s+\bif and only if\b\s+([A-Z~]+)', r'\1 <-> \2', s, flags=re.IGNORECASE)
    s = re.sub(r'\s+', '', s)
    return s

# Step 6: Define function to get sympy expression from input
# Determine if input is logic or sentence, parse accordingly, and create sympy expression.
def get_expr(input_str: str):
    if is_logic_input(input_str):
        logic_str = input_str.upper()
    else:
        logic_str = parse_sentence_to_logic(input_str)
    try:
        expr = sympify(logic_str, locals=locals_dict)
        return expr
    except Exception as e:
        raise ValueError(f"Could not parse input '{input_str}' to logic '{logic_str}': {e}")

# Step 7: Define function to convert sympy expression back to natural sentence
# Recursively build sentence based on operator type.
def to_natural_sentence(expr) -> str:
    if getattr(expr, "is_Atom", False):
        return prop_map.get(str(expr), str(expr))
    func = expr.func
    args = expr.args
    if func == And:
        return " and ".join(to_natural_sentence(arg) for arg in args)
    elif func == Or:
        return " or ".join(to_natural_sentence(arg) for arg in args)
    elif func == Not:
        return f"it is not the case that {to_natural_sentence(args[0])}"
    elif func == Implies:
        return f"if {to_natural_sentence(args[0])} then {to_natural_sentence(args[1])}"
    elif func == Equivalent:
        return f"{to_natural_sentence(args[0])} if and only if {to_natural_sentence(args[1])}"
    else:
        raise ValueError("Unsupported logical operator")

# Step 8: Define function to get variables from expression
# Extract and sort free symbols.
def get_vars(expr):
    return sorted(expr.free_symbols, key=str)

# Step 9: Define function to generate truth table
# Generate all combinations of truth values and evaluate expression.
def get_truth_table(expr):
    vars_list = get_vars(expr)
    table = []
    for values in product([True, False], repeat=len(vars_list)):
        sub = dict(zip(vars_list, values))
        result = bool(expr.subs(sub))
        row = list(values) + [result]
        table.append(row)
    return vars_list, table

# Step 10: Define function to print truth table
# Format and print the table with headers.
def print_truth_table(vars_list, table) -> None:
    header = [str(v) for v in vars_list] + ["Result"]
    line = " | ".join(header)
    print(line)
    print("-" * len(line))
    for row in table:
        print(" | ".join(str(r) for r in row))

# Step 11: Define base Term class for first-order logic
# Use dataclass for immutable Term.
@dataclass(frozen=True)
class Term:
    pass

# Step 12: Define Function (Fn) subclass of Term
# For constants and functions with arguments.
@dataclass(frozen=True)
class Fn(Term):
    name: str
    args: Tuple['Term', ...] = ()
    def __repr__(self) -> str:
        if len(self.args) == 0:
            return self.name
        return f"{self.name}({', '.join(map(repr, self.args))})"

# Step 13: Define Variable (Var) subclass of Term
# For variables prefixed with '?' in representation.
@dataclass(frozen=True)
class Var(Term):
    name: str
    def __repr__(self) -> str:
        return "?" + self.name

# Step 14: Define substitution function
# Apply substitution dictionary to terms recursively.
def subst(t: Term, s: Dict[str, Term]) -> Term:
    if isinstance(t, Var):
        return s.get(t.name, t)
    elif isinstance(t, Fn):
        return Fn(t.name, tuple(subst(arg, s) for arg in t.args))
    else:
        raise ValueError("Invalid term")

# Step 15: Define occurs_check for unification
# Check if variable occurs in term.
def occurs_check(x: Var, t: Term) -> bool:
    if isinstance(t, Var):
        return x.name == t.name
    elif isinstance(t, Fn):
        return any(occurs_check(x, arg) for arg in t.args)
    else:
        raise ValueError("Invalid term")

# Step 16: Define most general unifier (mgu)
# Implement unification algorithm with occurs check.
def mgu(t1: Term, t2: Term) -> Optional[Dict[str, Term]]:
    l1: List[Term] = [t1]
    l2: List[Term] = [t2]
    s: Dict[str, Term] = {}
    while len(l1) > 0:
        s1 = l1.pop()
        s2 = l2.pop()
        if isinstance(s1, Var):
            if s1 == s2:
                continue
            if occurs_check(s1, s2):
                return None
            l1 = [subst(term, {s1.name: s2}) for term in l1]
            l2 = [subst(term, {s1.name: s2}) for term in l2]
            s[s1.name] = s2
        elif isinstance(s2, Var):
            if occurs_check(s2, s1):
                return None
            l1 = [subst(term, {s2.name: s1}) for term in l1]
            l2 = [subst(term, {s2.name: s1}) for term in l2]
            s[s2.name] = s1
        elif isinstance(s1, Fn) and isinstance(s2, Fn):
            if s1.name != s2.name or len(s1.args) != len(s2.args):
                return None
            l1.extend(s1.args)
            l2.extend(s2.args)
        else:
            return None
    return s

# Step 17: Define Clause class
# Clauses with negative and positive literals.
@dataclass(frozen=True)
class Clause:
    neg: Tuple[Term, ...]
    pos: Tuple[Term, ...]
    def __repr__(self) -> str:
        return f"{self.neg} ⊢ {self.pos}"

# Step 18: Define function to compute resolvents
# Generate new clauses by resolving two clauses.
def compute_resolvents(c1: Clause, c2: Clause) -> List[Clause]:
    res: List[Clause] = []
    for lit1 in c1.pos:
        for lit2 in c2.neg:
            sigma = mgu(lit1, lit2)
            if sigma is None:
                continue
            new_neg = tuple(subst(l, sigma) for l in c1.neg) + tuple(subst(l, sigma) for l in c2.neg if l != lit2)
            new_pos = tuple(subst(l, sigma) for l in c1.pos if l != lit1) + tuple(subst(l, sigma) for l in c2.pos)
            new_clause = Clause(new_neg, new_pos)
            res.append(new_clause)
    return res

# Step 19: Define freshen function for terms
# Rename variables to avoid clashes.
def freshen(t: Term) -> Term:
    if isinstance(t, Var):
        return Var(t.name + "'")
    elif isinstance(t, Fn):
        return Fn(t.name, tuple(freshen(arg) for arg in t.args))
    else:
        raise ValueError("Invalid term")

# Step 20: Define freshen_clause
# Apply freshen to all literals in clause.
def freshen_clause(c: Clause) -> Clause:
    return Clause(tuple(freshen(l) for l in c.neg), tuple(freshen(l) for l in c.pos))

# Step 21: Define check for empty clause
# Indicates contradiction.
def is_empty_clause(c: Clause) -> bool:
    return len(c.neg) == 0 and len(c.pos) == 0

# Step 22: Define resolution inference function
# Perform resolution to check for unsatisfiability.
def resolution_inference(clauses: Set[Clause]) -> bool:
    seen: Set[Clause] = set(clauses)
    current: List[Clause] = list(clauses)
    while current:
        new: List[Clause] = []
        for i in range(len(current)):
            for j in range(i, len(current)):
                c1 = current[i]
                c2 = freshen_clause(current[j])
                resolvents = compute_resolvents(c1, c2) + compute_resolvents(c2, c1)
                for r in resolvents:
                    if is_empty_clause(r):
                        return True
                    if r not in seen:
                        seen.add(r)
                        new.append(r)
        if not new:
            break
        current = new
    return False

# Step 23: Define example inputs for propositional logic
# List of sentences and logic expressions to test.
inputs = [
    "it is raining and it is cold",
    "if it is raining then it is cold",
    "P & Q",
    "~P | Q"
]

# Step 24: Run propositional logic examples
# For each input, get expression, generate sentence, and print truth table.
for input_str in inputs:
    print(f"\nInput: {input_str}")
    try:
        expr = get_expr(input_str)
        print(f"Logic: {expr}")
        sentence = to_natural_sentence(expr)
        print(f"Generated Sentence: {sentence}")
        vars_list, table = get_truth_table(expr)
        print("Truth Table:")
        print_truth_table(vars_list, table)
    except ValueError as e:
        print(e)

# Step 25: Set up first-order logic example
# Define variables, constants, and clauses for contradiction test.
x = Var("x")
a = Fn("a")
clause1 = Clause((Fn("P", (x,)),), (Fn("Q", (x,)),))
clause2 = Clause((), (Fn("P", (a,)),))
clause3 = Clause((Fn("Q", (a,)),), ())

# Step 26: Run resolution inference
# Check if the set of clauses is unsatisfiable.
clauses = {clause1, clause2, clause3}
is_unsat = resolution_inference(clauses)
print(f"\nIs unsatisfiable (contradiction): {is_unsat}")



Input: it is raining and it is cold
Logic: P & Q
Generated Sentence: it is raining and it is cold
Truth Table:
P | Q | Result
--------------
True | True | True
True | False | False
False | True | False
False | False | False

Input: if it is raining then it is cold
Could not parse input 'if it is raining then it is cold' to logic 'P->Q': Sympify of expression 'could not parse 'P->Q'' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 1)

Input: P & Q
Logic: P & Q
Generated Sentence: it is raining and it is cold
Truth Table:
P | Q | Result
--------------
True | True | True
True | False | False
False | True | False
False | False | False

Input: ~P | Q
Logic: Q | ~P
Generated Sentence: it is cold or it is not the case that it is raining
Truth Table:
P | Q | Result
--------------
True | True | True
True | False | False
False | True | True
False | False | True

Is unsatisfiable (contradiction): False
